Code to simulate the winning utility model (Moral Strategy Model) at parametrizations throughout the model's parameter space (theta-phi space). These simulations are used later to draw strategy boundaries in the parameter space.

In [3]:
%matplotlib inline
import os, glob, sys, warnings, scipy
warnings.filterwarnings('always') # To filter numpy dtype warning

import numpy as np
import pandas as pd
from scipy import stats as ss
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('/project/3014018.02/analysis_mri/'+
                'DataSharingCollection/Code/1.Behavior-FitModels/Functions/')
import choiceModels, costFunctions, penalizedModelFit

base_dir = '/project/3014018.02/analysis_mri/DataSharingCollection/'

## Define functions

In [2]:
def simulate_data(theta=.25, phi=0):
    investment = np.arange(1,11,1)
    all_out = pd.DataFrame()
    for multiplier in [2,4,6]:
        out = pd.DataFrame({'Investment':investment})
        out['Predicted'] = np.array([choiceModels.MP_model_ppSOE(x,multiplier,4*x/2,theta,phi)
                                     for x in out['Investment']]).flatten()
        out['Multiplier'] = multiplier
        out['Theta'] = theta
        out['Phi'] = phi
        out['Expectation'] = 4*investment/2
        all_out = all_out.append(out)
    return all_out

## Prep simulation space

In [7]:
# Create theta-phi space
precision = 100
decimals = int(np.log10(precision)+1)
phi = np.around(np.arange(-0.1,0.10001,0.2/precision).astype('float'),decimals=decimals)#[-.1,0,.1]
theta = np.around(np.arange(0,.50001,.5/precision).astype('float'),decimals=decimals)#[0,.1,.2,.3,.4,.5,.6]
phiLen = len(phi)
print phi[0:10]
print phi[-10:]
print phiLen
thetaLen = len(theta)
print thetaLen
phi[np.floor(len(phi)/2).astype(int)]=0
# print phi
# print theta
thetaPhi = np.repeat(theta,phiLen)
thetaPhi = pd.DataFrame(thetaPhi)
thetaPhi.columns = ['theta']
thetaPhi['phi'] = np.tile(phi,thetaLen)
# print thetaPhi
print len(thetaPhi)

[-0.1   -0.098 -0.096 -0.094 -0.092 -0.09  -0.088 -0.086 -0.084 -0.082]
[0.082 0.084 0.086 0.088 0.09  0.092 0.094 0.096 0.098 0.1  ]
101
101
10201


In [20]:
thetaPhi.to_csv(os.path.join(base_dir,
                        'Results/2.Behavior-ClusterModel/ThetaPhi_%i-by-%i.csv'%(
                        thetaLen,phiLen)))

## Draw 101x101 simulations

In [34]:
# Run simulations
trialSet = pd.read_csv(os.path.join(base_dir,'Data/1.TaskBehavior/trialSet.csv'),index_col=0)
uniqueTrialSet = trialSet.drop_duplicates().reset_index(drop=True)
simulations = np.zeros((len(thetaPhi),uniqueTrialSet.shape[0]))
simulations_long = np.zeros((len(thetaPhi),trialSet.shape[0]))
for i in range(len(thetaPhi)):
    if np.mod(i,100) == 0:
        print i,
    theta = thetaPhi.iloc[i,0] #Watch out here: take the right index depending on the 'outer-inner' structure of thetaPhi
    phi = thetaPhi.iloc[i,1]
    singleSim = simulate_data(theta=theta,phi=phi)
    simulations[i,:] = singleSim['Predicted']
    simulations_long[i,:] = trialSet.merge(singleSim,on=['Investment','Multiplier'])['Predicted'] # Expand to full trial set
print simulations.shape
print simulations_long.shape

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 10100 10200 (10201, 30)
(10201, 76)


In [35]:
pd.DataFrame(simulations).to_csv(os.path.join(base_dir,
                        'Results/2.Behavior-ClusterModel/ModelSimulations_%i-by-%i.csv'%(
                        thetaLen,phiLen)))

In [36]:
pd.DataFrame(simulations_long).to_csv(os.path.join(base_dir,
                        'Results/2.Behavior-ClusterModel/ModelSimulations_long_%i-by-%i.csv'%(
                        thetaLen,phiLen)))